In [1]:
%load_ext autoreload
import numpy as np
# from netCDF4 import Dataset,num2date
# import glob
# from matplotlib.colors import LogNorm
# import matplotlib
# # matplotlib.use('Agg')
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# from matplotlib.colors import LogNorm
# from matplotlib import cm
# from subprocess import call
# from scipy import stats,signal
# import matplotlib.colorbar as cbar
# import matplotlib.ticker as ticker
# from scipy import interpolate
# import scipy
# from sklearn.metrics import r2_score
# import pickle
# from scipy.interpolate import griddata
import xarray as xr
from metpy.units import units

from util import spectra_analysis as spectra
from util import calc

In [7]:
%autoreload 2
file = "/glade/derecho/scratch/sturbeville/DPSCREAM_simulations/dpscreamv0_RCE_SMALL.lp05.P3_hom_t_atm_iulog_debug/run/dpscreamv0_RCE_SMALL.lp05.P3_hom_t_atm_iulog_debug.eam.h0.2000-01-01-00000.nc"

ds = spectra.open_w_dataarray(file)
ds

NameError: name 'pf' is not defined

In [5]:
attrex_ds = spectra.load_attrex_twp()
attrex_ds

<xarray.Dataset> Size: 240kB
Dimensions:     (wavelength: 5001)
Coordinates:
  * wavelength  (wavelength) float64 40kB inf 100.0 50.0 ... 0.02001 0.02 0.02
Data variables:
    p05         (wavelength) float64 40kB 0.006258 0.1532 ... 5.508e-08
    p25         (wavelength) float64 40kB 0.03987 0.3886 ... 3.711e-06 1.573e-06
    p50         (wavelength) float64 40kB 0.2017 0.8685 ... 1.338e-05 7.154e-06
    p75         (wavelength) float64 40kB 0.3811 1.91 ... 4.028e-05 2.092e-05
    p95         (wavelength) float64 40kB 0.8689 3.546 ... 0.0001533 7.525e-05
Attributes:
    long_name:  ATTREX_spectrum_detrended_100km_TWP_only
    history:    generated by Rachel Atlas using ATTREX campaign data

In [25]:
# variable names for each file for latitude, longitude,
# and vertical wind
lat_vars='lat'
lon_vars='lon'
w_vars='wa'

# #this is only for FV3
# lat_bnd_vars=['lat_bnds','lat_bnds','lat_bnds','clat_bnds']
# lon_bnd_vars=['lon_bnds','lon_bnds','lon_bnds','clon_bnds']

latres=3.33 * units.km
lonres=3.33 * units.km
dlon = spectra_util.calc_geodesic_distance((lat0,lon0),(lat0,lon1))
dlat = spectra_util.calc_geodesic_distance((lat0,lon0),(lat1,lon0))
nlon = dlon/lonres
nlat = dlat/latres

# Interpolate to a grid much finer than the real resolution
# to capture all variability
near_native_reg_lat = np.linspace(lat0,lat1,nlat+1)
near_native_reg_lon = np.linspace(lon0,lon1,nlon+1)
near_native_reg_latlon = np.meshgrid(near_native_reg_lat, near_native_reg_lat)

w_ds = spectra_util.load_w_dataarray(model_file_name)
model_unstructured_lat = 

#Indices of the closest level in each model to 14.2 km
#hinds = height indices
lev_index

#These will be used for ICON, which must be interpolated 
#to a rectangular grid

lon1d=np.arange(143,153.01,0.01)
lat1d=np.arange(-5,5.01,0.01)
lat_interp=np.zeros((1001,1001))
lon_interp=np.zeros((1001,1001))

for i in range(1001):
  lat_interp[i,:]=lat1d
  lon_interp[:,i]=lon1d

In [26]:
#Loop through four different models
j=0
model=models[j]

#load the model file
w_data=Dataset(w_dir+model)

#load latitude and logitude
lat=w_data.variables[lat_vars[j]][:]
lon=w_data.variables[lon_vars[j]][:]

#I am building a 2D grid from the vector of latitudes
#and longitudes reported from FV3
#change_points are the places where the vector will be "cut"
#to form strands
# if model_names[j]=='FV3':
#   lat_bnds=w_data.variables[lat_bnd_vars[j]][:]
#   lon_bnds=w_data.variables[lon_bnd_vars[j]][:]
#   change_points=np.squeeze(np.where(lat_bnds[:-1,0]-\
#                                     lat_bnds[1:,1]!=0.0))

#   #Not all strands are the same length so we have to account for this

#   xlen=np.size(change_points)+1 #Number of strands I cut the vector into
#   ylen=np.min(np.diff(change_points)) #Number of samples in the smalesst strand

#   minlen=np.min([xlen,ylen])

#   #Make square arrays to hold the data, we will cut the strands as needed later
#   w_reshape=np.zeros((minlen,minlen))
#   lon_reshape=np.zeros((minlen,minlen))
#   lat_reshape=np.zeros((minlen,minlen))
#   fv3_shape=np.shape(lon_reshape)

#Get latitude and longitude from ICON
if model_names[j]=='SCREAM':
    lat=lat
    lon=lon
    
    icon_coors=np.zeros((len(lon),2))
    icon_coors[:,0]=lon
    icon_coors[:,1]=lat

#tinds = time indices to include in average (3-hourly resolution)
#Allow 48 hours for spinup (begin at tind=16) and cut out times that are missing 
#or bad (tind=177 through tind=184) for one model (I forget which one...)
# tinds=np.concatenate((np.arange(16,177),np.arange(184,319)))
# tlen=tinds.size
tlen=200
k=0

#Loop through times to include
for tind in range(tlen): 
    print(tind, end=', ')
    #Interpolate ICON to rectanguar grid of fine resolution
    w_icon=w_data.variables[w_vars[j]][tind][hinds[j]]   
    w=griddata(icon_coors,w_icon,(lon_interp,lat_interp),
               method='nearest')
    
    lat_reshape=lat1d
    lon_reshape=lon1d
    w_reshape=np.zeros((1001,1001))
    w_reshape[:]=w[:]
    
    xlen=np.shape(w_reshape)[0]
    ylen=np.shape(w_reshape)[1]
    
    #Make sure that we take spectra over square grids so that 
    #we see the same range of wavelengths for each dimension
    minlen=np.min([xlen,ylen])
    
    w_reshape=w_reshape[:minlen,:minlen]
    
    if k==0:
      #The first time around the loop, create an array
      #to store power spectral density of w
      w_20hz_psd=np.zeros((tlen,minlen+minlen,\
                    len(scipy.signal.welch(w_reshape[0,:],\
                    detrend='linear',nperseg=10000)[0])))

    #Take spectra seperately over latitude and longitude and store them
    #I used to plot them seperately but I found that they were similar
    #so now I combine them
    for x in range(minlen):
      f_20hz_psd,w_20hz_psd[k,x] = scipy.signal.welch(\
                                         w_reshape[x,:],\
                                         detrend='linear',\
                                         nperseg=100000)
    
    for y in range(minlen):
      f_20hz_lf_lon,w_20hz_psd[k,minlen+y] = scipy.signal.welch(\
                                                w_reshape[:,y],\
                                                detrend='linear',\
                                                nperseg=100000)
    
    k=k+1

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 

In [27]:
#Plot spectra
fig,axes=plt.subplots(1,figsize=(4,4)) 

#Plot observations that were loaded from a python library earlier
axes.loglog(xaxis,median,'k')
axes.loglog(xaxis,p05,'k--')
axes.loglog(xaxis,p95,'k--')
axes.fill_between(xaxis,p25,p75,color='black',alpha=.25)

#Plot model sepctra
#The horizontal resolution divided by the fequency gives you the wavelength (on the x-axis) 
#Take statistics over all times and all latitudes and longitudes
axes.loglog(latres[j]/f_20hz_psd,np.median(w_20hz_psd,axis=(0,1)),'b')
axes.loglog(latres[j]/f_20hz_psd,np.quantile(w_20hz_psd,.05,axis=(0,1)),'b--')
axes.loglog(latres[j]/f_20hz_psd,np.quantile(w_20hz_psd,.95,axis=(0,1)),'b--')
axes.fill_between(latres[j]/f_20hz_psd,np.quantile(w_20hz_psd,\
  .25,axis=(0,1)),np.quantile(w_20hz_psd,.75,axis=(0,1)),\
       color='blue',alpha=.25)
axes.set_xlabel('Wavelength (km)')
axes.set_xlim(10,100)
axes.set_ylim(1e-7,10)
axes.set_xticks(np.arange(10,101,10))
axes.set_xticklabels(np.arange(10,101,10))
axes.set_xscale('log')
axes.set_yscale('log')
axes.set_ylabel(r"Power (m$^2$ s$^{-3}$)")
lev = int(w_data.variables['lev'][hinds[j]])
axes.set_title(f'DYAMOND 2 {model_names[j]} ({lev} hPa)')
plt.savefig(f'w_spectra_{model_names[j]}_h{hinds[j]}.png',dpi=600)
plt.show()

/tmp/ipykernel_231560/3564343792.py:13: RuntimeWarning: divide by zero encountered in divide
  axes.loglog(latres[j]/f_20hz_psd,np.median(w_20hz_psd,axis=(0,1)),'b')
/tmp/ipykernel_231560/3564343792.py:14: RuntimeWarning: divide by zero encountered in divide
  axes.loglog(latres[j]/f_20hz_psd,np.quantile(w_20hz_psd,.05,axis=(0,1)),'b--')
/tmp/ipykernel_231560/3564343792.py:15: RuntimeWarning: divide by zero encountered in divide
  axes.loglog(latres[j]/f_20hz_psd,np.quantile(w_20hz_psd,.95,axis=(0,1)),'b--')
/tmp/ipykernel_231560/3564343792.py:16: RuntimeWarning: divide by zero encountered in divide
  axes.fill_between(latres[j]/f_20hz_psd,np.quantile(w_20hz_psd,\
